In [ ]:
from __init__ import *

In [ ]:
directory_path = "../output/signal/"

kraft.path.path(directory_path)

In [ ]:
def surprise(table):

    matrix = table.to_numpy()

    assert (0 <= matrix).all()

    vector_1 = np.sum(matrix, axis=0)

    assert (0 < vector_1).all()

    for vector_0 in matrix:

        is_ = 0 < vector_0

        vector_0_is = vector_0[is_]

        vector_1_is = vector_1[is_]

        for i, number in enumerate(
            kraft.information.get_kld(
                vector_0_is / vector_0_is.sum(), vector_1_is / vector_1_is.sum(),
            )
        ):

            vector_0[i] = number

In [ ]:
for i, data in enumerate(SETTING["model_data_"]):

    table = pd.read_csv(data["file_path"], sep="\t", index_col=0)

    table.columns.name = data["axis_1_name"]

    surprise(table)

    kraft.table.summarize(table)

    table_negative = table.clip(upper=0)

    table_positive = table.clip(lower=0)

    axis = 1

    standard_deviation = 2

    table_negative = table_negative.loc[
        kraft.table.get_extreme_label_(
            table_negative.median(axis=axis), "<", standard_deviation=standard_deviation
        ),
        :,
    ]

    table_positive = table_positive.loc[
        kraft.table.get_extreme_label_(
            table_positive.median(axis=axis), ">", standard_deviation=standard_deviation
        ),
        :,
    ]

    if SETTING["signal"] == "-+":

        table_negative.index = (
            "-{}".format(label) for label in negative.index.to_numpy()
        )

        table_positive.index = (
            "+{}".format(label) for label in positive.index.to_numpy()
        )

        table_signal = pd.concat((-table_negative, table_positive))

    elif SETTING["signal"] == "-":

        table_signal = table_negative

    elif SETTING["signal"] == "+":

        table_signal = table_positive

    table_signal.index.name = table.index.name

    kraft.table.summarize(table_signal)

    table_signal.to_csv("{}{}.tsv".format(directory_path, i), sep="\t")